In [1]:
from time import sleep
from kafka import KafkaProducer
import json
import requests

In [2]:
producer = KafkaProducer(bootstrap_servers=['localhost:9093'],api_version=(0,10,1))

In [3]:
allData = []
for i in range(0, 100):
    urlMovies = "https://api.themoviedb.org/3/movie/"+str(i)+"?api_key=2baa0a850281337979fde9e4883df677"
    responseMovies = requests.request("GET", urlMovies)
    if(responseMovies.status_code == 200):
        urlReviews = "https://api.themoviedb.org/3/movie/"+str(i)+"/reviews?api_key=2baa0a850281337979fde9e4883df677"
        responseReviews = requests.request("GET", urlReviews)
        if(responseReviews.status_code == 200):
            genres = []
            content = []
            moviesData = json.loads(responseMovies.text)
            reviewsData = json.loads(responseReviews.text)
            for value in range(0,len(moviesData["genres"])):
                genres.append(moviesData["genres"][value]["name"])
            for value in range(0,len(reviewsData["results"])):
                if(value < 10):
                    content.append(reviewsData["results"][value]["content"])
            genre = ','.join(genres)
            review = ','.join(content)
            api_data_dict = {}
            api_data_dict['id'] = moviesData['id']
            api_data_dict['imdb_id'] = moviesData['imdb_id']
            api_data_dict['budget'] = moviesData['budget']
            api_data_dict['genre'] = genre
            api_data_dict['title'] = moviesData['title']
            api_data_dict['overview'] = moviesData['overview']
            api_data_dict['popularity'] = moviesData['popularity']
            api_data_dict['release_date'] = moviesData['release_date']
            api_data_dict['runtime'] = moviesData['runtime']
            api_data_dict['vote_average'] = moviesData['vote_average']
            api_data_dict['vote_count'] = moviesData['vote_count']
            api_data_dict['review'] = review
            allData.append(api_data_dict)
            #allData.append([moviesData["id"], moviesData["imdb_id"], moviesData["budget"], genres,moviesData["title"], moviesData["overview"], moviesData["popularity"], moviesData["release_date"],moviesData["runtime"], moviesData["vote_average"], moviesData["vote_count"],content])

In [4]:
#def sorters(sorter):
#    return sorter[6]
#allData.sort(key=sorters)

In [5]:
print(allData)

[{'id': 2, 'imdb_id': 'tt0094675', 'budget': 0, 'genre': 'Drama,Crime,Comedy', 'title': 'Ariel', 'overview': "Taisto Kasurinen is a Finnish coal miner whose father has just committed suicide and who is framed for a crime he did not commit. In jail, he starts to dream about leaving the country and starting a new life. He escapes from prison but things don't go as planned...", 'popularity': 9.264, 'release_date': '1988-10-21', 'runtime': 73, 'vote_average': 6.8, 'vote_count': 112, 'review': ''}, {'id': 3, 'imdb_id': 'tt0092149', 'budget': 0, 'genre': 'Drama,Comedy', 'title': 'Shadows in Paradise', 'overview': 'An episode in the life of Nikander, a garbage man, involving the death of a co-worker, an affair and much more.', 'popularity': 8.438, 'release_date': '1986-10-17', 'runtime': 72, 'vote_average': 7.3, 'vote_count': 118, 'review': "Released in 1986, Aki Kaurismaki's <i>Varjoja paratiisissa</i> (Shadows in Paradise) is one of the Finnish filmmaker's earliest efforts, and it stands as

In [6]:
print(len(allData))

68


In [7]:
print(type(allData))

<class 'list'>


In [8]:
for dict_data in allData:
    producer.send('tmdb-movies',json.dumps(dict_data).encode("utf-8"))
    sleep(5)